In [ ]:
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd
import mofax as mofa
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from palettable import wesanderson as wes


import warnings
warnings.filterwarnings("ignore")



In [ ]:
data_dir = "./../data/TCGA-BRCA/"
mod = ["mRNA", "DNAm" , "RPPA"]

In [ ]:
data = {}
for omic in mod : 
   with open(f"{data_dir}{omic}.pkl", "rb") as f:  # 'rb' = read binary
    data[omic] = pickle.load(f) 

In [ ]:
dna = data['DNAm']['expr']
rna = data['mRNA']['expr']
rppa = data['RPPA']['expr']

all_samples = dna.index.union(rna.index).union(rppa.index)

In [ ]:
dna = dna.reindex(all_samples)
rna = rna.reindex(all_samples)
rppa = rppa.reindex(all_samples)

In [ ]:
mods = {
    "dna": sc.AnnData(dna),
    "rna": sc.AnnData(rna),
    "rppa": sc.AnnData(rppa)
}

print(mods['dna'].obs.shape)
print(mods['rna'].obs.shape)
print(mods['rppa'].obs.shape)
print(mods["dna"].obs[:5])
print("\n")
print(mods['dna'].var.shape)
print(mods['rna'].var.shape)
print(mods['rppa'].var.shape)
print(mods['dna'].var[:5])
print("\n")
print(mods['dna'].X.shape)
print(mods['rna'].X.shape)
print(mods['rppa'].X.shape)
print(mods['dna'].X[:5])

print(mods['dna'].obs_names.shape)

In [ ]:
mdata = mu.MuData(mods)

In [ ]:
index = mdata.obs_names  # same as all_samples

meta_dna  = data['DNAm']['meta'].reindex(index)
meta_rna  = data['mRNA']['meta'].reindex(index)
meta_rppa = data['RPPA']['meta'].reindex(index)

meta = pd.concat([meta_dna, meta_rna, meta_rppa], axis=1)
meta = meta.loc[:, ~meta.columns.duplicated()]
mdata.obs = meta


In [ ]:
mdata

## One time training MOFA with whole datasets

In [ ]:
# don't need training anymore
# mu.tl.mofa(mdata, 
#            use_obs='union',
#            n_factors=15,
#            convergence_mode='medium',
#            outfile="exports/omics_union.hdf5")

In [ ]:
model = mofa.mofa_model("exports/omics_union.hdf5")
model

In [ ]:
mofa.plot_r2(model, x='View', vmax=15)

In [ ]:
x = mdata.obsm["X_mofa"]

In [ ]:
type(index)

In [ ]:
y1 = data['mRNA']['meta']['paper_BRCA_Subtype_PAM50'].reindex(index) 
y2 = data['DNAm']['meta']['paper_BRCA_Subtype_PAM50'].reindex(index)
y3 = data['RPPA']['meta']['paper_BRCA_Subtype_PAM50'].reindex(index)

y = y1.combine_first(y2).combine_first(y3)
y = y.to_numpy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)
print(y_encoded[:5])
print(type(x))

In [ ]:
from sklearn.model_selection import GridSearchCV
log_reg = LogisticRegression(max_iter=1000)
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "penalty": ["l2"],
    "multi_class": ["multinomial"]
}
grid = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    cv=5,                       # 5-fold CV
    scoring="accuracy",
    n_jobs=-1                   # use all CPU cores
)

grid.fit(x, y_encoded)
print("Best parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)


## Importing the model from the exported file

In [ ]:
import mofax as mofa

model = mofa.mofa_model("exports/omics_union.hdf5")


In [ ]:
factors_df = model.get_factors(df=True)  
factors_df.shape

## Feature reduction before training the MOFA model

In [ ]:
from sklearn.decomposition import PCA
from scipy import sparse

In [ ]:
views_expr = {
    "dna": data["DNAm"]["expr"],   # samples x CpGs
    "rna": data["mRNA"]["expr"],   # samples x genes
    "rppa": data["RPPA"]["expr"]   # samples x proteins
}

all_samples = mdata.obs_names  # or use the 'all_samples' you defined earlier


In [ ]:
mods_pca = {}

for view_name, expr_df in views_expr.items():
    # expr_df has only the samples measured in this omic, no NaNs yet
    X_raw = expr_df.to_numpy()     # shape: n_view_samples x n_features

    # up to 400 PCs, but not more than the number of features
    n_components = min(400, X_raw.shape[1])

    pca = PCA(
        n_components=n_components,
        svd_solver="randomized",
        random_state=0,
    )
    X_scores = pca.fit_transform(X_raw)  # n_view_samples x n_components

    # Put scores in a DataFrame with sample index
    pcs_df = pd.DataFrame(
        X_scores,
        index=expr_df.index,
        columns=[f"{view_name}_PC{i+1}" for i in range(n_components)]
    )

    # Reindex to the full sample set (union of samples across omics)
    # This introduces NaNs *after* PCA, which MOFA can handle as missing values.
    pcs_df = pcs_df.reindex(all_samples)

    # Create AnnData from the PC scores
    ad_pca = sc.AnnData(pcs_df.to_numpy())
    ad_pca.obs_names = pcs_df.index
    ad_pca.var_names = pcs_df.columns

    mods_pca[view_name] = ad_pca
    

In [ ]:
# New MuData with PCA-compressed views
mdata_pca = mu.MuData(mods_pca)

# Carry over sample metadata
mdata_pca.obs = mdata.obs.copy()

In [ ]:
mdata_pca

In [ ]:
mu.tl.mofa(
    mdata_pca,
    use_obs="union",
    n_factors=10,
    convergence_mode="medium",
    outfile="exports/omics_union_pca400_k10.hdf5"
)

In [ ]:
# Load the new model and extract factor scores (n_samples x 10)
model_pca = mofa.mofa_model("exports/omics_union_pca400_k10.hdf5")
factors_pca = model_pca.get_factors(df=True)
print("PCA-based MOFA factors shape:", factors_pca.shape)

In [ ]:
print(factors_pca.shape)

print(type(factors_pca))

x_pca = factors_pca.to_numpy()
print(x_pca[:5])

y1 = data['mRNA']['meta']['paper_BRCA_Subtype_PAM50'].reindex(index) 
y2 = data['DNAm']['meta']['paper_BRCA_Subtype_PAM50'].reindex(index)
y3 = data['RPPA']['meta']['paper_BRCA_Subtype_PAM50'].reindex(index)

y = y1.combine_first(y2).combine_first(y3)
y = y.to_numpy()
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)
print(y_encoded[:5])

y_pca_encoded = y_encoded

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=1000)
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "penalty": ["l2"],
    "multi_class": ["multinomial"]
}
grid = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    cv=5,                       # 5-fold CV
    scoring="accuracy",
    n_jobs=-1                   # use all CPU cores
)

grid.fit(x_pca, y_pca_encoded)
print("Best parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)